In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/travel-recommendation-datasets/travel_destination_dataset.csv


In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install gradio scikit-learn pandas

In [3]:
df= pd.read_csv('/kaggle/input/travel-recommendation-datasets/travel_destination_dataset.csv')

In [4]:
df.head(5)

,season,travel_month,budget,trip_duration_days,group_size,companion_type,interest,origin_city,origin_country,destination_name,destination_country
0,Spring,March,11649,1,2,Couple,Hiking,London,UK,Kruger National Park,South Africa
1,Winter,December,40573,9,2,Family,City Tour,Toronto,Canada,Rome,Italy
2,Autumn,October,7575,4,3,Solo,Adventure,London,UK,Santorini,Greece
3,Summer,August,43601,9,10,Family,Snow,São Paulo,Brazil,Kruger National Park,South Africa
4,Spring,May,6676,3,7,Couple,Cultural,Toronto,Canada,Machu Picchu,Peru


In [5]:
print(df.shape)
print(df.head())
print(df.dtypes)

(7000, 11)
   season travel_month  budget  trip_duration_days  group_size companion_type  \
0  Spring        March   11649                   1           2         Couple   
1  Winter     December   40573                   9           2         Family   
2  Autumn      October    7575                   4           3           Solo   
3  Summer       August   43601                   9          10         Family   
4  Spring          May    6676                   3           7         Couple   

    interest origin_city origin_country      destination_name  \
0     Hiking      London             UK  Kruger National Park   
1  City Tour     Toronto         Canada                  Rome   
2  Adventure      London             UK             Santorini   
3       Snow   São Paulo         Brazil  Kruger National Park   
4   Cultural     Toronto         Canada          Machu Picchu   

  destination_country  
0        South Africa  
1               Italy  
2              Greece  
3        South 

In [6]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [7]:
X = df[['season','travel_month','budget','trip_duration_days','group_size','companion_type','interest']]       


In [8]:
y_name    = df['destination_name']
y_country = df['destination_country']

In [9]:
le_name    = LabelEncoder().fit(y_name)
le_country = LabelEncoder().fit(y_country)
y = pd.DataFrame({
    'name':    le_name.transform(y_name),
    'country': le_country.transform(y_country)
})

In [10]:
numeric_feats = ['budget','trip_duration_days','group_size']
cat_feats     = ['season','travel_month','companion_type','interest']

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_feats),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_feats),
])

In [11]:
model = Pipeline([
    ('pre', preprocessor),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42)))
])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y['name']
)

In [13]:
model.fit(X_train, y_train)


Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['budget',
                                                   'trip_duration_days',
                                                   'group_size']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['season', 'travel_month',
                                                   'companion_type',
                                                   'interest'])])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))])

In [14]:
model.score(X,y)

0.8057142857142857

In [15]:
from sklearn.metrics import classification_report, accuracy_score

y_pred = model.predict(X_test)

In [16]:
model.score(X_train,y_train)

1.0

In [17]:
print("Name accuracy:", accuracy_score(y_test['name'], y_pred[:,0]))
print(classification_report(
    y_test['name'], y_pred[:,0],
    target_names=le_name.classes_
))

Name accuracy: 0.045714285714285714
                      precision    recall  f1-score   support

           Amsterdam       0.01      0.01      0.01        70
                Bali       0.07      0.09      0.08        77
               Banff       0.12      0.12      0.12        69
             Bangkok       0.04      0.03      0.03        64
           Bora Bora       0.04      0.04      0.04        71
           Cape Town       0.02      0.01      0.02        68
                 Goa       0.08      0.08      0.08        73
Kruger National Park       0.04      0.03      0.03        69
               Kyoto       0.04      0.03      0.03        64
              Lisbon       0.04      0.05      0.04        65
        Machu Picchu       0.04      0.03      0.03        71
            Maldives       0.02      0.03      0.02        73
            New York       0.01      0.01      0.01        70
               Paris       0.05      0.06      0.06        68
           Reykjavik       0.02  

In [18]:
print("Country accuracy:", accuracy_score(y_test['country'], y_pred[:,1]))
print(classification_report(
    y_test['country'], y_pred[:,1],
    target_names=le_country.classes_
))

Country accuracy: 0.05285714285714286
                  precision    recall  f1-score   support

       Australia       0.00      0.00      0.00        71
          Canada       0.08      0.07      0.08        69
          France       0.06      0.06      0.06        68
French Polynesia       0.05      0.04      0.04        71
          Greece       0.09      0.10      0.09        71
         Iceland       0.02      0.01      0.02        69
           India       0.07      0.07      0.07        73
       Indonesia       0.08      0.09      0.08        77
           Italy       0.04      0.04      0.04        70
           Japan       0.04      0.03      0.03        64
        Maldives       0.02      0.03      0.03        73
     Netherlands       0.04      0.04      0.04        70
            Peru       0.02      0.01      0.02        71
        Portugal       0.01      0.02      0.01        65
    South Africa       0.11      0.14      0.12       137
           Spain       0.07      

In [19]:
import joblib

joblib.dump({
    'model': model,
    'le_name': le_name,
    'le_country': le_country
}, 'travel_dest_predictor.pkl')


['travel_dest_predictor.pkl']

In [20]:
import shutil
shutil.move('travel_dest_predictor.pkl', '/kaggle/working/travel_dest_predictor.pkl')


'/kaggle/working/travel_dest_predictor.pkl'

In [28]:
import gradio as gr
import joblib
import pandas as pd

In [29]:
artifacts = joblib.load('travel_dest_predictor.pkl')
model    = artifacts['model']
le_name  = artifacts['le_name']
le_country = artifacts['le_country']

In [30]:
def predict(season, travel_month, budget, trip_duration_days,
            group_size, companion_type, interest):
    df_in = pd.DataFrame([{
        'season': season,
        'travel_month': travel_month,
        'budget': float(budget),
        'trip_duration_days': int(trip_duration_days),
        'group_size': int(group_size),
        'companion_type': companion_type,
        'interest': interest,
    }])
    pred = model.predict(df_in)[0]
    dest_name    = le_name.inverse_transform([pred[0]])[0]
    dest_country = le_country.inverse_transform([pred[1]])[0]
    # return two separate strings, in the same order as your outputs list
    return dest_name, dest_country


In [31]:
 inputs = [
     gr.Dropdown(choices=list(df['season'].unique()), label="Season"),
     gr.Dropdown(choices=list(df['travel_month'].unique()), label="Travel Month"),
     gr.Number(label="Budget"),
     gr.Number(label="Trip Duration (days)"),
     gr.Number(label="Group Size"),
     gr.Dropdown(choices=list(df['companion_type'].unique()), label="Companion Type"),
     gr.Dropdown(choices=list(df['interest'].unique()), label="Interest"),
 ]
outputs = [
    gr.Textbox(label="Predicted Destination Name"),
    gr.Textbox(label="Predicted Destination Country"),
]


In [32]:
demo = gr.Interface(fn=predict,
                    inputs=inputs,
                    outputs=outputs,
                    title="Travel Destination Recommender",
                    description="Enter your trip details to get a suggested destination.")
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://619d052e471e9396d6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
